# Exercise 1
## Task 2 - Motion Detection and Counting Cars 

The program will detect **6** vehicles in ***"Traffic_Laramie_1.mp4"*** and **4** vehicles in ***"Traffic_Laramie_2.mp4"*** going from the city’s downtown to the city centre.

#### installing OpenCV

In [1]:
!pip install opencv-python
!pip install numpy
!pip install tabulate

#### importing libraries 

In [2]:
import os
import cv2
import numpy as np
import pandas as pd
from tabulate import tabulate

### Motion Detection Code for Two Videos
My code is written with small descriptive functions that have limited purpose to follow separation of concerns principals. Markdown cells were used to explain what each function does and inline comments were used to add additional information. 

#### START - code was written based on module materials, independed research and documentation. Please see [Code References](#codereferences) section for all links.  

#### sets up video from path by using VideoCapture, and sets up background subtraction using MOG2 and warm up with first 50 frames

In [3]:
# START - code was written based on module materials, independed research and documentation. Please see Code References section for all links. 
# https://docs.opencv.org/master/de/de1/group__video__motion.html
def setup_video(file_path):   
    print(f"Evaluating video: {os.path.basename(file_path)}")

    # setting up video with helper function get_video which gets video by using cv2.videoCapture
    video_from_file = get_video(file_path)
    if not video_from_file.isOpened():
        print(f"Oh, no! Video {os.path.basename(file_path)} could not open. Please check if it exists in the path.")
        exit()
    else:
        print(f"The video {os.path.basename(file_path)} was successfully opened!")
    
    background_subtraction = setup_background_model()
    
     # warm up phase with first 50 frames for background subtraction stabilization
     # https://github.com/shrutitharmia/motion-detector
     # https://github.com/shrutitharmia/motion-detector/blob/main/motion_detector.py
    warm_up_frames = 50
    for _ in range(warm_up_frames):
        frame_found, frame = video_from_file.read()
        if not frame_found:
            return None, None
        background_subtraction.apply(frame)
    
    return video_from_file, background_subtraction

# END - code was written based on module materials, independed research and documentation. Please see Code References section for all links. 

#### sets up a background subtraction model using MOG2 

In [4]:
# START - code was written based on module materials, independed research and documentation. Please see Code References section for all links. 
# https://docs.opencv.org/4.x/d7/d7b/classcv_1_1BackgroundSubtractorMOG2.html
# https://docs.opencv.org/4.x/d1/dc5/tutorial_background_subtraction.html
# https://docs.opencv.org/4.x/de/de1/group__video__motion.html
# https://answers.opencv.org/question/182167/what-does-the-history-of-this-function-createbackgroundsubtractormog2-means/
def setup_background_model():
    return cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=False)

# END - code was written based on module materials, independed research and documentation. Please see Code References section for all links. 

#### gets video from path path

In [5]:
# START - code was written based on module materials, independed research and documentation. Please see Code References section for all links. 
# https://note.nkmk.me/en/python-opencv-videocapture-file-camera/
# https://docs.opencv.org/4.x/d8/dfe/classcv_1_1VideoCapture.html
def get_video(file_path):
    return cv2.VideoCapture(file_path)

# END - code was written based on module materials, independed research and documentation. Please see Code References section for all links. 

#### calculates duration of video in secs

In [6]:
# START - code was written based on module materials, independed research and documentation. Please see Code References section for all links. 
# https://www.tutorialspoint.com/get-video-duration-using-opencv-python
# https://stackoverflow.com/questions/31472155/python-opencv-cv2-cv-cv-cap-prop-frame-count-get-wrong-numbers
# https://github.com/opencv/opencv/issues/24000
def calculate_duration_in_seconds(video_from_file):
    frames_per_seconds = video_from_file.get(cv2.CAP_PROP_FPS)
    total_frames = video_from_file.get(cv2.CAP_PROP_FRAME_COUNT)
    video_duration_in_seconds = total_frames / frames_per_seconds
    return video_duration_in_seconds

# END - code was written based on module materials, independed research and documentation. Please see Code References section for all links. 

#### obtains center of the vehicle’s bounds

In [7]:
# START - code was written based on module materials, independed research and documentation. Please see Code References section for all links. 
# https://github.com/ultralytics/ultralytics/issues/4212
# http://www.open3d.org/docs/release/python_api/open3d.geometry.OrientedBoundingBox.html
# https://imgaug.readthedocs.io/en/latest/_modules/imgaug/augmentables/bbs.html
# https://github.com/carla-simulator/carla/blob/master/PythonAPI/examples/client_bounding_boxes.py
# https://github.com/ultralytics/ultralytics/issues/2843
class Vehicle:
    def __init__(self, bounding_box):
        self.bounding_box = bounding_box
        self.since_last_update_frames = 0

    # calculates center of bounding box      
    def get_center(self):
        x, y, w, h = self.bounding_box
        return x + w // 2, y + h // 2 # enter of bounding box
    
# END - code was written based on module materials, independed research and documentation. Please see Code References section for all links. 

#### applies morphological transformation to mask

In [8]:
# START - code was written based on module materials, independed research and documentation. Please see Code References section for all links. 
# https://docs.opencv.org/4.x/d9/d61/tutorial_py_morphological_ops.html
# https://www.geeksforgeeks.org/python-opencv-morphological-operations/
# https://medium.datadriveninvestor.com/image-processing-using-python-open-cv-part-2-72f2b75918e7
# https://stackoverflow.com/questions/55288523/im-getting-error-with-morphological-mask
def morphological_transformation(foreground_mask):
    # processes mask by enhancing region of interest
    kernel_matrix = np.ones((5, 5), np.uint8)
    foreground_mask = cv2.morphologyEx(foreground_mask, cv2.MORPH_CLOSE, kernel_matrix)
    mask = np.zeros_like(foreground_mask)
    REGION_OF_INTEREST = np.array([[(300, 500), (350, 350), (400, 100), (400, 100), (400, 200), (500, 300)]], dtype=np.int32)
    cv2.fillPoly(mask, REGION_OF_INTEREST, 255)
    foreground_mask = cv2.bitwise_and(foreground_mask, mask)
    
    return foreground_mask # enhanced mask

# END - code was written based on module materials, independed research and documentation. Please see Code References section for all links. 

#### render one frame of video at the time and listen for 'q' key press to stop it

In [9]:
# START - code was written based on module materials, independed research and documentation. Please see Code References section for all links. 
# https://forum.opencv.org/t/how-to-use-waitkey-with-videocapture/10718
# https://answers.opencv.org/question/232169/closing-video-with-any-key/
# https://www.geeksforgeeks.org/python-opencv-waitkey-function/
def render_frame(frame):
    # renders one frame at the time and listen for 'q' key press to stop it
    cv2.imshow('frame', frame)
    if cv2.waitKey(30) == ord('q'):
        return True
    return False

# END - code was written based on module materials, independed research and documentation. Please see Code References section for all links. 

#### analyzes each frame of video, tracks and detects vehicles to return all detected vehicles

In [10]:
# START - code was written based on module materials, independed research and documentation. Please see Code References section for all links. 
# https://stackoverflow.com/questions/26344036/python-opencv-background-subtraction
# https://www.geeksforgeeks.org/python-opencv-background-subtraction/
def analyze_frames(video_from_file, background_subtraction):
    tracked_vehicles = []
    all_detected_vehicles = []

    while True:
        frame_found, frame = video_from_file.read()
        if not frame_found:
            break

        foreground_mask = background_subtraction.apply(frame)
        foreground_mask = morphological_transformation(foreground_mask)

        tracked_vehicles, all_detected_vehicles = detect_track_vehicles_in_current_frame(foreground_mask, tracked_vehicles, all_detected_vehicles, frame)

        if render_frame(frame):
            break

    return all_detected_vehicles

# END - code was written based on module materials, independed research and documentation. Please see Code References section for all links. 

#### analyzes one video, detects and tracks vehicles

In [11]:
# START - code was written based on module materials, independed research and documentation. Please see Code References section for all links. 
# https://cs.gmu.edu/~kosecka/cs482/code-examples/opencv-python/OpenCV_Python.pdf
# https://analyticsindiamag.com/getting-started-with-opencv-in-python/
# https://www.javatpoint.com/opencv-videocapture
# https://stackoverflow.com/questions/54794177/cv2-error-opencv3-4-3-error-after-finish-playing-video
def analyze_video(file_path):
    video_from_file, background_subtraction = setup_video(file_path)
    if video_from_file is None or background_subtraction is None:
        return

    all_detected_vehicles = analyze_frames(video_from_file, background_subtraction)
    
    unique_vehicle_count = len(all_detected_vehicles)
    video_duration_in_seconds = calculate_duration_in_seconds(video_from_file)
    vehicles_per_second = unique_vehicle_count / video_duration_in_seconds
    vehicles_per_minute = vehicles_per_second * 60
    
    video_from_file.release()
    cv2.destroyAllWindows()
    
    # return dict with video results
    return {
        "Video name": os.path.basename(file_path),
        "Total number of cars": unique_vehicle_count,
        "Cars per minute": vehicles_per_minute
    } 

# END - code was written based on module materials, independed research and documentation. Please see Code References section for all links. 

#### detects vehicles in frame by using contours

In [12]:
# START - code was written based on module materials, independed research and documentation. Please see Code References section for all links. 
# https://docs.opencv.org/4.x/d4/d73/tutorial_py_contours_begin.html
# https://learnopencv.com/contour-detection-using-opencv-python-c/
# https://gist.github.com/m3hrdadfi/664de37b216eb5c97affb4b3db331089
# https://stackoverflow.com/questions/68399140/python-cv2-find-contours
def detect_vehicles(foreground_mask):
    contours, _ = cv2.findContours(foreground_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    detected_vehicles = []
    WIDTH_CONSTRAINT = 100
    HEIGHT_CONSTRAINT = 45
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if w > WIDTH_CONSTRAINT and h > HEIGHT_CONSTRAINT:
            detected_vehicles.append((x, y, w, h))
    return detected_vehicles # returns boundting boxes which are identified vehicles

# END - code was written based on module materials, independed research and documentation. Please see Code References section for all links. 

#### matchs detected vehicles with presently tracked vehicles and handle mismatched vehicles 

In [13]:
# START - code was written based on module materials, independed research and documentation. Please see Code References section for all links. 
# https://thepythoncode.com/article/real-time-vehicle-tracking-and-counting-with-yolov8-opencv#google_vignette
# https://github.com/yehengchen/Object-Detection-and-Tracking/blob/master/OneStage/yolo/yolov3_sort/main.py
# https://carla.readthedocs.io/en/latest/tuto_G_bounding_boxes/
# https://github.com/jeongwhanchoi/object-detection
# https://github.com/pytorch/vision/issues/2740
# https://www.mdpi.com/1424-8220/19/19/4263
def track_and_match(detected_vehicles, tracked_vehicles, all_detected_vehicles):
    # array of current vehicles
    current_vehicles = []
    # loops over bounding boxes of detected vehicles 
    for x, y, w, h in detected_vehicles:
        # calcutes center of bounding box of vehicle
        vehicle_center = (x + w // 2, y + h // 2)
        match_found = False # flag
        # loops over each tracked vehicles 
        for vehicle in tracked_vehicles:
            # calculates distance between center of tracked vehicle and center of detected vehicle
            distance = np.linalg.norm(np.array(vehicle.get_center()) - np.array(vehicle_center))
            # if less than max width and height, than it's a match
            if distance < max(w, h):  # max of width and height as constraint
                # tracked vehicle's bounding box is updated to match detected's bounding box
                vehicle.bounding_box = (x, y, w, h)
                # resets frame counter 
                vehicle.since_last_update_frames = 0
                match_found = True # flag that vehicle was found
                break
        # if not match, then identified vehicle is a new vehicle
        if not match_found:
            new_vehicle = Vehicle((x, y, w, h)) # new vehicle
            current_vehicles.append(new_vehicle) # adds it to the list of current vehicles
            all_detected_vehicles.append(new_vehicle) # adds it to list of all vehicles
    return current_vehicles, all_detected_vehicles

# END - code was written based on module materials, independed research and documentation. Please see Code References section for all links. 

#### draws rects around detected vehicle 

In [14]:
# START - code was written based on module materials, independed research and documentation. Please see Code References section for all links. 
# https://www.geeksforgeeks.org/python-opencv-cv2-rectangle-method/
def draw_detected_vehicles_rectangles(frame, detected_vehicles):
    for x, y, w, h in detected_vehicles:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
# END - code was written based on module materials, independed research and documentation. Please see Code References section for all links. 

#### updates frames since last counter for all tracked vehicles and removes them if no longer in frame

In [15]:
# START - code was written based on module materials, independed research and documentation. Please see Code References section for all links. 
# https://docs.python.org/3/library/copy.html
def update_and_remove_vehicles(tracked_vehicles):
    # loop over a copy of tracked vehicle list 
    for vehicle in tracked_vehicles.copy():
        # increment counter indicating how many frames it has been since the car was last updated
        vehicle.since_last_update_frames += 1
        # if vehicle not updated in over 30 frames, remove it from tracked vehicles
        if vehicle.since_last_update_frames > 30:
            tracked_vehicles.remove(vehicle)
            
# END - code was written based on module materials, independed research and documentation. Please see Code References section for all links. 

#### detects and track vehicles in current frame

In [16]:
# START - code was written based on module materials, independed research and documentation. Please see Code References section for all links. 
# https://www.programiz.com/python-programming/methods/list/extend
def detect_track_vehicles_in_current_frame(foreground_mask, tracked_vehicles, all_detected_vehicles, frame):
    # detects vehicles with foreground mask
    detected_vehicles = detect_vehicles(foreground_mask)
    # matchsand unmatches detected with tracked vehicles
    current_vehicles, all_detected_vehicles = track_and_match(detected_vehicles, tracked_vehicles, all_detected_vehicles)
    # draws rects for detected vehicles
    draw_detected_vehicles_rectangles(frame, detected_vehicles)
    # uses extend() to combine tracked vehicles list with new detected vehicles 
    tracked_vehicles.extend(current_vehicles)
    # updates and remove vehicles not seen for specified time  
    update_and_remove_vehicles(tracked_vehicles)
    # returns new list of tracked and detected vehicles
    return tracked_vehicles, all_detected_vehicles

# END - code was written based on module materials, independed research and documentation. Please see Code References section for all links. 

####  analyzes videos in specified path, every video gets analyzed for with logic in analyze_video() function which detects and tracks vehicles to return a list of results for each video

In [17]:
# START - code was written based on module materials, independed research and documentation. Please see Code References section for all links. 
# https://docs.python.org/3/library/os.path.html
def analyze_videos(file_paths):
    result_data_list = []
    # individual videos
    for file_path in file_paths:
        result = analyze_video(file_path)
        
        # adds video name to result
        if result is not None:  # make sure None results are not added
            result['Video name'] = os.path.basename(file_path)
            result_data_list.append(result)
    return result_data_list

# END - code was written based on module materials, independed research and documentation. Please see Code References section for all links. 

#### handles display of data in a table by using tabulate, df and pandas

In [18]:
# START - code was written based on module materials, independed research and documentation. Please see Code References section for all links. 
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_markdown.html
# https://stackoverflow.com/questions/18528533/pretty-printing-a-pandas-dataframe
def display_result_table(result_data_list):
    df = pd.DataFrame(result_data_list)
    print(tabulate(df, headers='keys', tablefmt='grid', showindex=False))

# END - code was written based on module materials, independed research and documentation. Please see Code References section for all links. 

#### calls analyze_videos and display_result_table functions which handle vehicle detection and tracking for indicated video paths and displaying results in a table

In [19]:
# START - code was written based on module materials, independed research and documentation. Please see Code References section for all links. 

def main():
        file_paths = ['assets/Traffic_Laramie_1.mp4', 'assets/Traffic_Laramie_2.mp4']
        result_data_list = analyze_videos(file_paths)
        display_result_table(result_data_list)

if __name__ == "__main__":
    main()
    
# END - code was written based on module materials, independed research and documentation. Please see Code References section for all links. 

Evaluating video: Traffic_Laramie_1.mp4
The video Traffic_Laramie_1.mp4 was successfully opened!
Evaluating video: Traffic_Laramie_2.mp4
The video Traffic_Laramie_2.mp4 was successfully opened!
+-----------------------+------------------------+-------------------+
| Video name            |   Total number of cars |   Cars per minute |
+=======================+========================+===================+
| Traffic_Laramie_1.mp4 |                      6 |           2.02338 |
+-----------------------+------------------------+-------------------+
| Traffic_Laramie_2.mp4 |                      4 |           2.27101 |
+-----------------------+------------------------+-------------------+


#### END - code was written based on module materials, independed research and documentation. Please see [Code References](#codereferences) section for all links.  

<a id='codereferences'></a>
## Code References
- https://docs.opencv.org/master/de/de1/group__video__motion.html
- https://github.com/shrutitharmia/motion-detector
- https://github.com/shrutitharmia/motion-detector/blob/main/motion_detector.py
- https://docs.opencv.org/4.x/d7/d7b/classcv_1_1BackgroundSubtractorMOG2.html
- https://docs.opencv.org/4.x/d1/dc5/tutorial_background_subtraction.html
- https://docs.opencv.org/4.x/de/de1/group__video__motion.html
- https://answers.opencv.org/question/182167/what-does-the-history-of-this-function-createbackgroundsubtractormog2-means/
- https://note.nkmk.me/en/python-opencv-videocapture-file-camera/
- https://docs.opencv.org/4.x/d8/dfe/classcv_1_1VideoCapture.html
- https://www.tutorialspoint.com/get-video-duration-using-opencv-python
- https://stackoverflow.com/questions/31472155/python-opencv-cv2-cv-cv-cap-prop-frame-count-get-wrong-numbers
- https://github.com/opencv/opencv/issues/24000
- https://github.com/ultralytics/ultralytics/issues/4212
- http://www.open3d.org/docs/release/python_api/open3d.geometry.OrientedBoundingBox.html
- https://imgaug.readthedocs.io/en/latest/_modules/imgaug/augmentables/bbs.html
- https://github.com/carla-simulator/carla/blob/master/PythonAPI/examples/client_bounding_boxes.py
- https://github.com/ultralytics/ultralytics/issues/2843
- https://docs.opencv.org/4.x/d9/d61/tutorial_py_morphological_ops.html
- https://www.geeksforgeeks.org/python-opencv-morphological-operations/
- https://medium.datadriveninvestor.com/image-processing-using-python-open-cv-part-2-72f2b75918e7
- https://stackoverflow.com/questions/55288523/im-getting-error-with-morphological-mask
- https://forum.opencv.org/t/how-to-use-waitkey-with-videocapture/10718
- https://answers.opencv.org/question/232169/closing-video-with-any-key/
- https://www.geeksforgeeks.org/python-opencv-waitkey-function/
- https://stackoverflow.com/questions/26344036/python-opencv-background-subtraction
- https://www.geeksforgeeks.org/python-opencv-background-subtraction/
- https://cs.gmu.edu/~kosecka/cs482/code-examples/opencv-python/OpenCV_Python.pdf
- https://analyticsindiamag.com/getting-started-with-opencv-in-python/
- https://www.javatpoint.com/opencv-videocapture
- https://stackoverflow.com/questions/54794177/cv2-error-opencv3-4-3-error-after-finish-playing-video
- https://docs.opencv.org/4.x/d4/d73/tutorial_py_contours_begin.html
- https://learnopencv.com/contour-detection-using-opencv-python-c/
- https://gist.github.com/m3hrdadfi/664de37b216eb5c97affb4b3db331089
- https://stackoverflow.com/questions/68399140/python-cv2-find-contours
- https://thepythoncode.com/article/real-time-vehicle-tracking-and-counting-with-yolov8-opencv#google_vignette
- https://github.com/yehengchen/Object-Detection-and-Tracking/blob/master/OneStage/yolo/yolov3_sort/main.py
- https://carla.readthedocs.io/en/latest/tuto_G_bounding_boxes/
- https://github.com/jeongwhanchoi/object-detection
- https://github.com/pytorch/vision/issues/2740
- https://www.mdpi.com/1424-8220/19/19/4263
- https://www.geeksforgeeks.org/python-opencv-cv2-rectangle-method/
- https://docs.python.org/3/library/copy.html
- https://www.programiz.com/python-programming/methods/list/extend
- https://docs.python.org/3/library/os.path.html
- https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_markdown.html
- https://stackoverflow.com/questions/18528533/pretty-printing-a-pandas-dataframe
